# **Import Statements:**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
BASE_PATH = "/content/drive/My Drive/CS196Project/Group6"
%cd $BASE_PATH

!pip install -r ./finetuning/requirements.txt

from datasets import load_dataset
import torch

Get all required code for finetuning.

In [ ]:
%%bash
cd "/content/drive/My Drive/CS196Project/Group6/finetuning"

wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/finetune.py
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/callbacks.py
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/utils.py
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/sentence_splitter.py
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/lightning_base.py
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/requirements.txt

In [26]:
torch.cuda.empty_cache()
torch.cuda.is_available()

True

# **Load Datasets:**

 TODO: format newsroom


In [4]:
newsroom = load_dataset('newsroom', data_dir='/content/drive/My Drive/CS196Project/newsroom')

Using custom data configuration default


Dataset newsroom downloaded and prepared to /root/.cache/huggingface/datasets/newsroom/default/1.0.0/4b405ccd64e15f685065870ea563a1e6a034d1bd269a5427f40146d81549095e. Subsequent calls will reuse this data.


Create Custom Dataset:

In [24]:
import re

%cd "/content/drive/My Drive/CS196Project/Group6"

class CreateDataset:
  def __init__(self, folder_path: str, split_ratio=(2, 1, 1)):
    assert len(split_ratio) == 3
    assert type(split_ratio) == tuple
    # split_ratio = (train, test, val)

    assert type(folder_path) == str
    assert folder_path != None and len(folder_path) > 0

    if folder_path[:-1] != "/":
      folder_path += "/"

    self.data_path = data_path

    self.iter_counter = 0
    self.split_counter = 0
    self.train_cut = split_ratio[0]
    self.test_cut = self.train_cut + split_ratio[1]
    self.val_cut = self.test_cut + split_ratio[2]

    train_source = open("{}train.source".format(folder_path), "w+")
    train_target = open("{}train.target".format(folder_path), "w+")
    test_source = open("{}test.source".format(folder_path), "w+")
    test_target = open("{}test.target".format(folder_path), "w+")
    val_source = open("{}val.source".format(folder_path), "w+")
    val_target = open("{}val.target".format(folder_path), "w+")

    self.sources = [train_source, test_source, val_source]
    self.targets = [train_target, test_target, val_target]

  def create_data_files(self, data_path: str, finished=False) -> None:
    assert type(data_path) == str
    assert data_path != None and len(data_path) > 0

    lines = self._get_prepared_lines(data_path)

    for example in lines:
      if not len(example):  # ignore any emtpy text
        continue
      
      source, target = self._next_entry()

      data = data.replace("[SEP]", "")
      text, summary = data.split("[SUMM]")

      source.write("{}\n".format(text))
      target.write("{}\n".format(summary))

    if finished:
      self._finish()

  def _get_prepared_lines(self, data_path: str) -> list:
    text_file = open(self.data_path, "r").read()
    text_file = text_file.replace("\n", "")
    # strip text of all newlines, so we can control where they get places later on
    # newlines are used to separate entries in train/test/validation datasets
    text_file = re.sub(r'\.(?=[^ ])', '. ', text_file)
    # replace a period followed immediately with a character with a period and a space

    split_by_separation = text_file.split("[TEXT]")
    
    return split_by_separation

  def _finish(self) -> None:
    for f in self.sources:
      f.close()
    
    for f in self.targets:
      f.close()
      
  def _next_entry(self) -> tuple:
    if self.iter_counter == self.val_cut:
      self.split_counter = 0
      self.iter_counter = 0
    elif self.iter_counter == self.train_cut or self.iter_counter == self.test_cut:
      self.split_counter += 1
      
    self.iter_counter += 1
    return self.sources[self.split_counter], self.targets[self.split_counter]

Create the text training, testing, and validation files.

In [ ]:
dataset = CreateDataset("finetuning/lecture_transcripts/")
dataset.create_data_files("lecture_training.txt", finished=True)

# **Finetuning Model**

In [27]:
%%bash
BASE_PATH="/content/drive/My Drive/CS196Project/Group6"

cd "/content/drive/My Drive/CS196Project/Group6"
cd finetuning
pwd

# Unused Alternative Parameters:
    # --do_predict \
    # --eval_batch_size=1 \

    # --freeze_embeds \
    # --fp16 \


export OUTPUT_DIR="example_output"
export DATA_DIR="lecture_transcripts"

python finetune.py \
    --learning_rate=3e-5 \
    --gpus 1 \
    --do_train \
    --n_val 1000 \
    --val_check_interval 0.1 \
    --data_dir=$DATA_DIR \
    --train_batch_size=1 \
    --output_dir=$OUTPUT_DIR \
    --overwrite_output_dir \
    --freeze_encoder \
    --num_train_epochs 6 \
    --max_target_length=60 \
    --val_max_target_length=60 \
    --test_max_target_length=100 \
    --model_name_or_path sshleifer/distilbart-xsum-12-6
    "$@"

/content/drive/My Drive/CS196Project/Group6
/content/drive/My Drive/CS196Project/Group6/finetuning
Epoch 0:   8%|▊         | 1/12 [00:00<00:05,  2.16it/s, loss=4.519, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 0:  25%|██▌       | 3/12 [00:57<02:53, 19.24s/it, loss=4.302, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 0:  42%|████▏     | 5/12 [00:59<01:23, 12.00s/it, loss=4.059, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 0:  58%|█████▊    | 7/12 [02:19<01:39, 19.94s/it, loss=3.955, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 0:  75%|███████▌  | 9/12 [02:21<00:47, 15.77s/it, loss=3.719, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 0:  92%|█████████▏| 11/12 [02:24<00:13, 13.11s/it, loss=3.404, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 1:   8%|▊         | 1/12 [00:00<00:05,  2.09it/s, loss=3.157, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 1:  25%|██▌       | 3/12 [00:02<00:08,  1.11it/s, loss=3.045, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 1:  42%|████▏     | 5/12 [00:0

2020-11-04 19:52:45.532218: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Downloading: 100%|██████████| 1.43k/1.43k [00:00<00:00, 1.92MB/s]
Downloading: 100%|██████████| 899k/899k [00:00<00:00, 32.8MB/s]
Downloading: 100%|██████████| 456k/456k [00:00<00:00, 29.6MB/s]
Downloading: 100%|██████████| 26.0/26.0 [00:00<00:00, 34.8kB/s]
Downloading: 100%|██████████| 611M/611M [00:15<00:00, 38.3MB/s]
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Checkpoint directory /content/drive/My Drive/CS196Project/Group6/finetuning/example_output exists and is not empty. With save_top_k=1, all files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed

# **Test Model**

Showing that the fine-tuning has an effect: 

1.   Summarization with stored fine-tuned model
2.   Summarization with default model used



In [30]:
from transformers import AutoModelForSeq2SeqLM, AutoModelWithLMHead
from transformers import AutoTokenizer, pipeline

%cd "/content/drive/My Drive/CS196Project/Group6"

device = 'cpu'
text = "DNA evidence has transformed forensic science and the criminal justice system. But what are the limits of using DNA as a tool in forensic science? To address this question we need to learn the basic structure of DNA and the rules of inheritance. Let’s start by simply defining a few terms, and we’ll start with the term heredity. Heredity can simply be defined as the passing of traits to offspring. We can think about this in 2 time scales—a short time scale over just 1 generation or a long time scale involving ancestors that go back millennia. Another term that we should define is phenotype. A measurable trait of an organism is often referred to as a phenotype. Here we list examples under different categories like disease or behavioral phenotypes. We can classify phenotypes into 2 types of traits: simple inherited traits and complex inherited traits.Traits of simple inheritance have the following characteristics: They are monogenic or controlled by only 1 gene. They are binary or discontinuous. And the expression of these traits is not greatly influenced by the environment. Simple inherited traits represent only a small fraction of all human traits."
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-xsum-12-6") 

# 1. Summarization with fine-tuned model 
model = AutoModelForSeq2SeqLM.from_pretrained(f'finetuning/example_output/best_tfmr')
model = model.to(device)
summarizer = pipeline('summarization', model=model, tokenizer=tokenizer)

print(summarizer(text))

# 2. Summarization with default model used before finetuning
model = AutoModelWithLMHead.from_pretrained("sshleifer/distilbart-xsum-12-6")
model.to(device)
summarizer = pipeline('summarization', model=model, tokenizer=tokenizer)

print(summarizer(text))

/content/drive/My Drive/CS196Project/Group6
[{'summary_text': ' The use of DNA as a tool in forensic science has been hailed as a game-changer.'}]


/usr/local/lib/python3.6/dist-packages/transformers/modeling_auto.py:825: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


[{'summary_text': 'DNA, or DNA, has been used to determine who is responsible for a crime.'}]
